In [1]:
import dash  #(version 1.12.0)
from dash.dependencies import Input, Output
import dash_table as dt
from dash import dcc
from dash import html
import plotly.express as px
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

import pandas as pd
import numpy as np

#Automcompletar rápido
%config IPCompleter.greedy=True

<ipython-input-1-196a4f49381f>:3: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


In [2]:
# Import the cleaned data (importing csv into pandas)
impact_target=0.2653
df = pd.read_csv("../data/final_data.csv")
#df=df[df['Churn']==0]

df = df[["customerID","Contract","tenure","TotalCharges","InternetService","pred","cluster"]]

# Creating an ID column name gives us more interactive capabilities
df.set_index('customerID', inplace=True, drop=False)

df=df[df["pred"]>=impact_target]
print(df.columns)

Index(['customerID', 'Contract', 'tenure', 'TotalCharges', 'InternetService',
       'pred', 'cluster'],
      dtype='object')


In [3]:
# # Cargamos las discretizaciones
# df_disc_tc=pd.read_csv("../data/discret_TotalCharges.csv")
# df_disc_tenure=pd.read_csv("../data/discret_tenure.csv")
# df_disc_contract=pd.read_csv("../data/discret_Contract.csv")


# # Limpiamos un poco las conversiones
# df_disc_tc.rename(columns={"Unnamed: 0":"intervalo"},inplace=True)
# df_disc_tenure.rename(columns={"Unnamed: 0":"intervalo"},inplace=True)
# df_disc_contract.rename(columns={"Unnamed: 0":"intervalo"},inplace=True)


# # Transformamos a diccionario para poder usar map
# disc_tc=df_disc_tc['intervalo'].to_dict()
# disc_tenure=df_disc_tenure['intervalo'].to_dict()
# disc_contract=df_disc_contract['index'].to_dict()


# # Reemplazamos por los valores originales de las discretizaciones
# df["TotalCharges_disc_corr"]= df["TotalCharges_disc_corr"].map(disc_tc)
# df["tenure_disc_corr"]= df["tenure_disc_corr"].map(disc_tenure)
# df["Contract"]= df["Contract"].map(disc_contract)

In [4]:
df['pred']=df['pred'].transform(lambda x: np.round(x,3))
df=df.sort_values(by='pred',ascending=False).reset_index(drop=True)

In [5]:
df.shape

(2845, 7)

In [6]:
df.head()

,customerID,Contract,tenure,TotalCharges,InternetService,pred,cluster
0,8098-LLAZX,Month-to-month,4,396.10,Fiber optic,0.907,4
1,7216-EWTRS,Month-to-month,1,100.80,Fiber optic,0.907,4
2,7024-OHCCK,Month-to-month,2,170.85,Fiber optic,0.907,4
3,2720-WGKHP,Month-to-month,2,181.70,Fiber optic,0.900,4
4,5150-ITWWB,Month-to-month,3,335.75,Fiber optic,0.900,4


# Grafico 1

- Clientes ordenados por scoring de mayor a menor vs score

In [7]:
df.head()

,customerID,Contract,tenure,TotalCharges,InternetService,pred,cluster
0,8098-LLAZX,Month-to-month,4,396.10,Fiber optic,0.907,4
1,7216-EWTRS,Month-to-month,1,100.80,Fiber optic,0.907,4
2,7024-OHCCK,Month-to-month,2,170.85,Fiber optic,0.907,4
3,2720-WGKHP,Month-to-month,2,181.70,Fiber optic,0.900,4
4,5150-ITWWB,Month-to-month,3,335.75,Fiber optic,0.900,4


In [8]:
dff = df.copy()
dff.reset_index(inplace=True)

fig1 = px.line(dff, 
               x='index',
               y='pred',
               height=350,
               title="Scores Ordenados")

# Grafico 2 y 4

- grafico 2: clientes ordenados en forma dec por score vs margenes (MN, ME)
- grafico 4: clientes ordenados en forma dec por score vs ROI

In [9]:
# Calculamos el margen esperado medio por cliente
# Introducimos una columna para el coste medio 

mar_medio = 1000
coste_medio = 300

df['ME']=df['pred']*mar_medio
df['MN']=df['ME']-coste_medio

# Creamos df_camp
df_camp=df.sort_values(by='MN',ascending=False)
df_camp.reset_index(drop=True,inplace=True)
df_camp.reset_index(inplace=True)

In [10]:
df_camp['INGRESOS_AGRE']=df_camp['MN'].cumsum()
df_camp['COSTES_AGRE']=(df_camp["index"]+1)*coste_medio
df_camp['ROI']= df_camp['INGRESOS_AGRE']-df_camp['COSTES_AGRE']

In [11]:
df_camp.head()

,index,customerID,Contract,tenure,TotalCharges,InternetService,pred,cluster,ME,MN,INGRESOS_AGRE,COSTES_AGRE,ROI
0,0,8098-LLAZX,Month-to-month,4,396.10,Fiber optic,0.907,4,907.0,607.0,607.0,300,307.0
1,1,7024-OHCCK,Month-to-month,2,170.85,Fiber optic,0.907,4,907.0,607.0,1214.0,600,614.0
2,2,7216-EWTRS,Month-to-month,1,100.80,Fiber optic,0.907,4,907.0,607.0,1821.0,900,921.0
3,3,2720-WGKHP,Month-to-month,2,181.70,Fiber optic,0.900,4,900.0,600.0,2421.0,1200,1221.0
4,4,4424-TKOPW,Month-to-month,2,196.75,Fiber optic,0.900,4,900.0,600.0,3021.0,1500,1521.0


In [12]:
# Grafico 3d de segmentacion
dff = df_camp.copy()
fig2 = px.scatter_3d(df, x='Contract', 
                    y='tenure', 
                    z='pred',
                    color='cluster', 
                    #size='pred', 
                    #size_max=100,
                    symbol='InternetService', 
                    opacity=0.5,
                    height=350)



In [13]:
ind_ROI_max = dff[dff['ROI']==dff['ROI'].max()]["index"].to_list()[0]
score_ROI_max=dff[dff['ROI']==dff['ROI'].max()]["pred"].to_list()[0]
ROI_max=dff["ROI"].max()

print(ind_ROI_max)
print(ROI_max)
print(score_ROI_max)

961
115792.0
0.601


In [14]:
dff = df_camp.copy()
fig4 = px.line(dff,
               x='index',
               y='ROI', 
               height=350,
               title="Grafico de ROI")


# Grafico 3

- Grafico de clientes vs ingresos agregados, costos agregados y ROI

In [15]:
# Creamos dos dataframe con los clientes que tengan un score mayor al doble y triple

score_x2 = impact_target * 2
score_x3 = impact_target * 3

bote_camp_x2=df[df['pred']>score_x2]
bote_camp_x3=df[df['pred']>score_x3]


In [16]:
bote_camp_x2.reset_index(inplace=True)
indice_x2=bote_camp_x2[len(bote_camp_x2)-1:]['index'].to_list()[0]

In [17]:
bote_camp_x3.reset_index(inplace=True)
indice_x3=bote_camp_x3[len(bote_camp_x3)-1:]['index'].to_list()[0]


In [18]:
score_indice_x3=bote_camp_x3[bote_camp_x3['index']==indice_x3]['pred'].to_list()[0]

In [19]:
indice_x2

1294

In [20]:
score_indice_x3

0.796

In [21]:
indice_x3

187

In [22]:
dff = df_camp.copy()
fig3 = px.line(dff, 
               x='index', 
               y=['INGRESOS_AGRE','COSTES_AGRE','ROI'],
               height=350,
               title="Grafico de Costos, Ingresos y ROI")

In [23]:
ind_cero=dff[dff['ROI']<=0]['index'].to_list()[0]
ind_cero

2051

In [24]:
data_graf=df_camp.copy()
data_graf=data_graf[['customerID','pred','cluster']]

# DASHBOARD

In [ ]:
# https://www.bootstrapcdn.com/bootswatch/
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )

template ='plotly_dark'
# 'ggplot2', plotly_dark, plotly_white, seaborn, plotly,presentation, ygridoff


# Layout section: Bootstrap (https://hackerthemes.com/bootstrap-cheatsheet/)
# ************************************************************************
app.layout = dbc.Container([


    dbc.Row([

         dbc.Col([
            dt.DataTable(id='tbl',
                         data=data_graf.to_dict('records'),
                         columns=[{"name": i, "id": i} for i in data_graf.columns],
                         editable=True,              # allow editing of data inside all cells
                         filter_action="native",     # allow filtering of data by user ('native') or not ('none')
                         sort_action="native",       # enables data to be sorted per-column by user or not ('none')
                         sort_mode="single",         # sort across 'multi' or 'single' columns
                         column_selectable="multi",  # allow users to select 'multi' or 'single' columns
                         #row_selectable="multi",     # allow users to select 'multi' or 'single' rows
                         #row_deletable=True,         # choose if user can delete a row (True) or not (False)
                         selected_columns=[],        # ids of columns that user selects
                         selected_rows=[],           # indices of rows that user selects
                         page_action="native",       # all data is passed to the table up-front or not ('none')
                         page_current=0,             # page number that user is on
                         page_size=15,                # number of rows visible per page
                         style_cell={             # ensure adequate header width when text is shorter than cell's text
                             #'backgroundColor': 'rgb(222, 245, 242)'
                             'minWidth': 60, 
                             'maxWidth': 60, 
                             'width': 60, 
                             'textAlign': 'center', 
                             'fontSize':10
                         },
                         style_data={                # overflow cells' content into multiple lines
                             'whiteSpace': 'normal',
                             'height': '10'
                         },
                         style_header={
                             'backgroundColor': 'rgb(222, 245, 242)',
                             'fontWeight': 'bold',
                             'fontSize':12
                         },
                         style_table={'height': '550px',
                                      'overflowY': 'auto'
                         })
             
         ],width=2, align='left',class_name="p-3 border bg-light"),
                  
        dbc.Col([
            
            dbc.Row([
                dcc.Graph(id='line-fig1', 
                          figure=fig1
                          .add_vrect(x0=0, 
                                     x1=indice_x2, 
                                     annotation_text="Zona de score x2", 
                                     annotation_position="top right",
                                     fillcolor="skyblue", 
                                     opacity=0.25, 
                                     line_width=0)
                           .add_hrect(y0=0, 
                                     y1=0.21, 
                                     annotation_text="Zona maximiza recall minimiza Precision", 
                                     annotation_position="bottom right",
                                     fillcolor="red", 
                                     opacity=0.2, 
                                     line_width=0)
                          .add_hrect(y0=0.21, 
                                     y1=0.37, 
                                     annotation_text="Zona maximiza precision minimiza Recall", 
                                     annotation_position="top right",
                                     fillcolor="yellow", 
                                     opacity=0.2, 
                                     line_width=0)
                          .update_layout(hovermode="x unified",
                                         showlegend=False,
                                         template=template,
                                         font_size=9,
                                         margin=dict(t=30, r=0, l=0, b=15)
                                         #paper_bgcolor='rgba(0,0,0,0)',
                                         #plot_bgcolor='rgba(0,0,0,0)',
                                         )
                          .update_traces(mode="lines")
                          .add_hline(y=score_ROI_max,
                                     line_width=1,
                                     line_dash="dash",
                                     line_color="skyblue")
                          .add_vline(x=ind_ROI_max,
                                     line_width=1,
                                     line_dash="dash",
                                     line_color="skyblue")
                          
                          
                         )
            ],justify='around',class_name="row gy-5"),
            
            dbc.Row([
                dcc.Graph(id='line-fig3', 
                          figure=fig2
                          .update_layout(margin=dict(l=0, r=0, b=0, t=0),
                                         hovermode="x unified",
                                         showlegend=False,
                                         template="plotly_dark",
                                         #width=600,
                                         #height=350,
                                         font_size=9)
                         )
            ],justify='around')
        ],width=5,align='center'),
        
        dbc.Col([
            dbc.Row([
                dcc.Graph(id='line-fig2', 
                          figure=fig3
                          .update_layout(hovermode="x unified",
                                         showlegend=False,
                                         template=template,
                                         font_size=9, 
                                         margin=dict(t=30, r=0, l=0, b=15))
                          .update_traces(mode="lines")
                          .add_vline(x=ind_cero,
                                     line_width=1,
                                     line_dash="dash",
                                     line_color="red")
                         )
            ], justify='around',class_name="row gy-5"),
            
            dbc.Row([
                dcc.Graph(id='line-fig4', 
                          figure=fig4
                          .update_layout(hovermode="x unified",
                                         showlegend=False,
                                         template=template,
                                         font_size=9, 
                                         margin=dict(t=30, r=0, l=0, b=15))
                          .update_traces(mode="lines")
                          .add_annotation(
                              text='<i>Retorno Optimo de la inversion en el maximo de la curva</i>',
                              xref="paper", 
                              yref="paper",
                              x=0.1, 
                              y=0.3,
                              showarrow=False)
                          .add_vline(x=ind_ROI_max,
                                     line_width=1,
                                     line_dash="dash",
                                     line_color="red")
                          )
            ],justify='around')
        ],width=5,align='center')
    ])

],fluid=True)




if __name__ == '__main__':
    app.run_server(debug=False)

    
# https://youtu.be/0mfIK8zxUds

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2021 20:05:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 20:05:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 20:05:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 20:05:44] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 20:05:44] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 20:05:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 20:05:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


# DataTable

In [29]:
# App layout
app = dash.Dash(__name__, prevent_initial_callbacks=True) # this was introduced in Dash version 1.12.0

# Sorting operators (https://dash.plotly.com/datatable/filtering)
app.layout = html.Div([
    dt.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True, "hideable": True}
            if i == "Contract" or i == "customerID"
            else {"name": i, "id": i, "deletable": True, "selectable": True}
            for i in df.columns
        ],
        data=df.to_dict('records'),  # the contents of the table
        editable=True,              # allow editing of data inside all cells
        filter_action="native",     # allow filtering of data by user ('native') or not ('none')
        sort_action="native",       # enables data to be sorted per-column by user or not ('none')
        sort_mode="single",         # sort across 'multi' or 'single' columns
        column_selectable="multi",  # allow users to select 'multi' or 'single' columns
        row_selectable="multi",     # allow users to select 'multi' or 'single' rows
        row_deletable=True,         # choose if user can delete a row (True) or not (False)
        selected_columns=[],        # ids of columns that user selects
        selected_rows=[],           # indices of rows that user selects
        page_action="native",       # all data is passed to the table up-front or not ('none')
        page_current=0,             # page number that user is on
        page_size=10,                # number of rows visible per page
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'minWidth': 60, 'maxWidth': 60, 'width': 60
        },
        style_cell_conditional=[    # align text columns to left. By default they are aligned to right
            {
                'if': {'column_id': c},
                'textAlign': 'center'
            } for c in ['customerID','Contract','tenure_disc_corr', 'TotalCharges_disc_corr']
        ],
        style_data={                # overflow cells' content into multiple lines
            'whiteSpace': 'normal',
            'height': 'auto'
        }
    ),

    html.Br(),
    html.Br(),
    html.Div(id='bar-container'),
    html.Div(id='choromap-container')

])


# -------------------------------------------------------------------------------------
# Highlight selected column
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# -------------------------------------------------------------------------------------


if __name__ == '__main__':
    app.run_server(debug=False) # Ojo aqui con debug.. En un editor de codigo se puede poner debug=True
    

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [06/Nov/2021 20:04:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2021 20:04:13] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2021 20:04:13] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2021 20:04:13] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2021 20:04:13] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
